!pip install pandas matplotlib prophet requests python-dotenv
|

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
from dotenv import load_dotenv
import os, requests

In [8]:
# Load dataset
df = pd.read_csv("C:\\Users\\ANIS\\Downloads\\Weather forcast ML\\weather.csv")

# Show actual column names
print("Columns:", df.columns.tolist())

# Convert datetime
df['date'] = pd.to_datetime(df['date'], errors="coerce", utc=True)

# Keep required columns
df_main = df[['date', 
              'Temperature (C)', 
              'Humidity', 
              'Wind Speed (km/h)', 
              'Visibility (km)', 
              'Rainfall', 
              'Pressure (millibars)']]

# Drop missing rows
df_main = df_main.dropna()

print("Shape after cleaning:", df_main.shape)
df_main.head()


Columns: ['date', 'Rainfall', 'Temperature (C)', 'Humidity', 'Wind Speed (km/h)', 'Visibility (km)', 'Pressure (millibars)']
Shape after cleaning: (96453, 7)


,date,Temperature (C),Humidity,Wind Speed (km/h),Visibility (km),Rainfall,Pressure (millibars)
0,2006-03-31 22:00:00+00:00,9.472222,0.89,14.1197,15.8263,15.501725,1015.13
1,2006-03-31 23:00:00+00:00,9.355556,0.86,14.2646,15.8263,8.833681,1015.63
2,2006-04-01 00:00:00+00:00,9.377778,0.89,3.9284,14.9569,17.186099,1015.94
3,2006-04-01 01:00:00+00:00,8.288889,0.83,14.1036,15.8263,13.977624,1016.41
4,2006-04-01 02:00:00+00:00,8.755556,0.83,11.0446,15.8263,1.974129,1016.51


In [9]:
# Parse with timezone info
df['date'] = pd.to_datetime(df['date'], utc=True, errors="coerce")

# Remove timezone, keep naive datetime
df['date'] = df['date'].dt.tz_convert(None)
df.head()

,date,Rainfall,Temperature (C),Humidity,Wind Speed (km/h),Visibility (km),Pressure (millibars)
0,2006-03-31 22:00:00,15.501725,9.472222,0.89,14.1197,15.8263,1015.13
1,2006-03-31 23:00:00,8.833681,9.355556,0.86,14.2646,15.8263,1015.63
2,2006-04-01 00:00:00,17.186099,9.377778,0.89,3.9284,14.9569,1015.94
3,2006-04-01 01:00:00,13.977624,8.288889,0.83,14.1036,15.8263,1016.41
4,2006-04-01 02:00:00,1.974129,8.755556,0.83,11.0446,15.8263,1016.51


In [10]:
def forecast_feature(df, column, periods=30):
    """
    Forecast a single feature using Prophet.
    """
    # Ensure datetime has no timezone
    df['date'] = pd.to_datetime(df['date'], errors="coerce").dt.tz_localize(None)

    df_temp = df[['date', column]].rename(columns={
        'date': 'ds',
        column: 'y'
    }).dropna()

    if df_temp.shape[0] < 2:
        raise ValueError(f"⚠️ Not enough data for {column}.")
    
    model = Prophet()
    model.fit(df_temp)

    # Future dates
    future = model.make_future_dataframe(periods=periods)
    forecast = model.predict(future)

    # Plot forecast
    fig = model.plot(forecast)
    plt.title(f"{column} Forecast")
    plt.show()

    return forecast


In [11]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path=r"C:\\Users\\ANIS\\Downloads\\Weather forcast ML\\.env")
API_KEY = os.getenv("OPENWEATHER_API_KEY")
print("API Key loaded?", bool(API_KEY))



API Key loaded? True


In [12]:
CITY = "Delhi"  # change as needed

if API_KEY:
    url = f"http://api.openweathermap.org/data/2.5/weather?q={CITY}&appid={API_KEY}&units=metric"
    try:
        res = requests.get(url)
        res.raise_for_status()  # raises error for bad responses
        data = res.json()
        print("🌤 Live Weather Data:", data)
    except requests.exceptions.RequestException as e:
        print("⚠️ Request failed:", e)
else:
    print("⚠️ Please add OPENWEATHER_API_KEY in your .env file")


🌤 Live Weather Data: {'coord': {'lon': 77.2167, 'lat': 28.6667}, 'weather': [{'id': 701, 'main': 'Mist', 'description': 'mist', 'icon': '50n'}], 'base': 'stations', 'main': {'temp': 27.05, 'feels_like': 30.77, 'temp_min': 27.05, 'temp_max': 27.05, 'pressure': 1004, 'humidity': 89, 'sea_level': 1004, 'grnd_level': 979}, 'visibility': 3000, 'wind': {'speed': 3.09, 'deg': 80}, 'clouds': {'all': 40}, 'dt': 1758144716, 'sys': {'type': 1, 'id': 9165, 'country': 'IN', 'sunrise': 1758155851, 'sunset': 1758200000}, 'timezone': 19800, 'id': 1273294, 'name': 'Delhi', 'cod': 200}
